In [1]:
# hello world
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
import string
import re
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.sql import functions as F
sc = pyspark.SparkContext()

In [2]:
from pyspark.sql import *

In [3]:
spark = SparkSession.builder.appName("Projec3")\
        .config("spark.some.config.option","some-value")\
        .getOrCreate()

In [4]:
df = spark.read.load("/home/cse587/CSE587/Project3/train.csv",sep=",",format="csv",inferSchema = True,header = True,escape ='"')

In [5]:
df.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- genre: string (nullable = true)



In [6]:
df.show() #formated correctly

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|
|24225279|                Sing|The story begins ...|           ['Drama']|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|
|20532852|Destination Meatball|A line of people ...|['Animation', 'Sh...|
|15401493|    Husband for Hire|Lola  a

In [7]:

#punc = string.punctuation
def removePunctuation(s):
    print("type of s: ",s)
    return re.sub(r'[^\w\s]',"",s)
#data = re.sub(r'[^\w\s]',"",data)

In [9]:
udf = UserDefinedFunction(lambda x: removePunctuation(x), StringType())
df = df.select(*[udf(column).alias("plot") if column == "plot" else column for column in df.columns])
#new_df = df.withColumn("plot",)
df.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov a hardwor...|['World cinema', ...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|
|  595909|   A Cry in the Dark|Seventhday Advent...|['Crime Fiction',...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|
| 1952976|          Dark Water|plot The film ope...|['Thriller', 'Dra...|
|24225279|                Sing|The story begins ...|           ['Drama']|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|
|20532852|Destination Meatball|A line of people ...|['Animation', 'Sh...|
|15401493|    Husband for Hire|Lola  a

In [10]:
genreMap = spark.read.load("/home/cse587/CSE587/Project3/mapping.csv",sep=",",format="csv",inferSchema = True,header = True,escape ='"')
genreMap = genreMap.withColumnRenamed('_c0','id')
genreMap = genreMap.withColumnRenamed('0','genre')
genreMap.show()

#this converts a column into a list for use inside encodeGenre() 
#since you cant perform computations on a dataframe inside a UDF
genreList = genreMap.select("genre").rdd.flatMap(lambda x: x).collect()

+---+----------------+
| id|           genre|
+---+----------------+
|  0|           Drama|
|  1|          Comedy|
|  2|    Romance Film|
|  3|        Thriller|
|  4|          Action|
|  5|    World cinema|
|  6|   Crime Fiction|
|  7|          Horror|
|  8| Black-and-white|
|  9|           Indie|
| 10|Action/Adventure|
| 11|       Adventure|
| 12|     Family Film|
| 13|      Short Film|
| 14|  Romantic drama|
| 15|       Animation|
| 16|         Musical|
| 17| Science Fiction|
| 18|         Mystery|
| 19| Romantic comedy|
+---+----------------+



In [17]:

def encodeGenre(s):
    #s : the value of col "genre" at some row
    retVal=""
    #s = re.sub(r'[^\\'']',"",s)
    #print("s: ",s)
    #print("type of s: ",type(s))
    
    #convert from string to list
    s = s[1:-1].split(", ")
    #print("type of s: ",type(s))
    #print(s)
    
    #remove single quotes, regex wasnt working for some reason
    temp=[]
    for x in s:
        #s.remove(x)
        x=x[1:-1]
#         print(x)
#         index = genreMap["0"==x]
#         print(index)
        temp.append(x)
    s=temp
    #print(s)
    
    #create one hot encoding, 
    for genre in genreList:
        if genre in s:
            retVal=retVal+"1"
        else:
            retVal=retVal+"0"
##### apparetnly it aint kosher to perform df operations inside a udf... so this hour of work was for nothing
#     t = genreMap.withColumn("encoding",F.when(genreMap.genre.isin(s),1).otherwise(0))
#     #t.show()
#     retVal = t.select("encoding").rdd.flatMap(lambda x: x).collect()
#     retVal = ''.join(map(str,retVal))
    return retVal
    
    

In [18]:
udf2 = UserDefinedFunction(lambda x: encodeGenre(x), StringType())
df = df.select(*[udf2(column).alias("genre") if column == "genre" else column for column in df.columns])
df.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov a hardwor...|10000100000000000000|
|31186339|    The Hunger Games|The nation of Pan...|10001000001000000100|
|20663735|          Narasimham|Poovalli Induchoo...|10001000000000001000|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|01000000000000000000|
|  595909|   A Cry in the Dark|Seventhday Advent...|10000110000000000000|
| 5272176|            End Game|The president is ...|10011000001000000000|
| 1952976|          Dark Water|plot The film ope...|10010001000000000000|
|24225279|                Sing|The story begins ...|10000000000000000000|
| 2462689|       Meet John Doe|Infuriated at bei...|11100000100000000001|
|20532852|Destination Meatball|A line of people ...|00000000000011010000|
|15401493|    Husband for Hire|Lola  a

In [ ]:
######NOT WORKING BELOW HERE, just trying some stuff out to create the term-document matrix

In [ ]:
text_split = df.select("plot").rdd.map(lambda line: line.split(" "))

In [ ]:
text_word = text_split.map(lambda word: (word,1))

In [ ]:
word_count = text_word.reduceByKey(lambda a, b: b+a)

In [ ]:
word_count.collect()